# **Welcome to the Notebook**

### Task 1 - Set up the project

Installing the needed modules.

In [37]:
!pip install openai==1.16.2 python-dotenv pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.21.0
    Uninstalling openai-2.21.0:
      Successfully uninstalled openai-2.21.0


In [1]:
!pip install -U httpx
!pip install -U openai

  Using cached openai-2.21.0-py3-none-any.whl.metadata (29 kB)
Using cached openai-2.21.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: openai
    Found existing installation: openai 1.16.2
    Uninstalling openai-1.16.2:
      Successfully uninstalled openai-1.16.2


Imporint the modules

In [2]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType

from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.clustering import KMeans
import plotly.express as px

Setup the OpenAI API

In [3]:
load_dotenv(dotenv_path='/content/apikey.env.txt')
APIKEY = os.getenv("APIKEY")

In [4]:
client = OpenAI(
    api_key=APIKEY,
)
client

Create a Spark session

In [5]:
spark = SparkSession.builder.appName("EcommerceProductRecommendation").getOrCreate()
spark

Loading the dataset

In [6]:
file_path = "/content/products_dataset.csv"
df= spark.read.csv(file_path, header=True, inferSchema=True, samplingRatio = 1)
df.show()

+----------+--------------------+--------------------+
|product_id|               title|         description|
+----------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|
|        P1|Turmode 30 ft. RP...|If you need more ...|
|        P2|Large Tapestry Bo...|Polyester cover r...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|
|        P9|Traditional Silve...|This transitional...|
|       P10|15 in. x 59 in. O...|Its easy to add a...|
|       P11|1 qt. #350F-7 Wil...|BEHR PREMIUM PLUS...|
|       P12|Anthracite Cordle...|BlindsAvenue ligh...|
|       P13|SlimGrip 78-Inch ...|Luverne SlimGrip ...|
|       P14|6 in. x 28 in. x ...|Our Rustic Collec...|
|       P1

List of 8 products recently viewed by the user.

In [7]:
recently_viewed_products = [
    'P316',
    'P333',
    'P1115',
    'P1691',
    'P1082',
    'P397',
    'P1441',
    'P1054',
]

### Task 2 - Prepare the dataset

Combine `title` and `description` Columns

In [8]:
df = df.withColumn("combined_text", concat_ws(" ", "title", "description"))
df.show()

+----------+--------------------+--------------------+--------------------+
|product_id|               title|         description|       combined_text|
+----------+--------------------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|Men's 3X Large Ca...|
|        P1|Turmode 30 ft. RP...|If you need more ...|Turmode 30 ft. RP...|
|        P2|Large Tapestry Bo...|Polyester cover r...|Large Tapestry Bo...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|16-Gauge-Sinks Ve...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|Men's Crazy Horse...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|Mariana 6 ft. Mul...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|5 gal. #650C-2 Po...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|7/8 in. x 4-1/2 i...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|Ring Gold Bar Car...|
|        P9|Traditional Silve...|This transitional...|Traditional Silve...|
|       P10|

get the combined_text column and convert it into a list

In [9]:
list_combined_text = df.select("combined_text").rdd.flatMap(lambda x: x).collect()
print(list_combined_text[:4])

["Men's 3X Large Carbon Heather Cotton/Polyester Rain Defender Paxton Heavyweight Hooded Zip-Front Sweatshirt This heavyweight, water-repellent hooded sweatshirt has a zip front for fast layering. ORIGINAL FIT. 13 oz., 75% cotton/25% polyester blend with Rain Defender durable water repellent. Attached, jersey-lined three-piece hood with drawcord closure. Antique-finish brass front zipper. Two front hand-warmer pockets have a hidden security pocket inside. Stretchable, spandex-reinforced rib-knit cuffs and waistband. Locker loop facilitates hanging.", "Turmode 30 ft. RP TNC Female to RP TNC Male Adapter Cable If you need more length between your existing wireless device and Hi-Gain Antenna, this is the product for you. It's compatible with most Wi-Fi Antennas, so it is easy for you to extend your wireless network. Just replace your existing cable that runs between your wireless device and Antenna and you're ready to use your network with extended range.", 'Large Tapestry Bolster Bed Pol

Use OpenAI text embedding model to create the vector embeddings.

In [10]:
response = client.embeddings.create(
    input= list_combined_text,
    model="text-embedding-3-small",
    dimensions= 512,
)
embedding_vectors = [data.embedding for data in response.data]
embedding_vectors[:2]

[[0.042664770036935806,
  0.02092622220516205,
  -0.013632948510348797,
  -0.0020650296937674284,
  0.003196326084434986,
  -0.03726103901863098,
  0.027195259928703308,
  0.07833647727966309,
  0.0549556165933609,
  -0.06283164769411087,
  0.04421878606081009,
  0.048315733671188354,
  -0.0678468719124794,
  0.02521742321550846,
  0.022550875321030617,
  0.06392651796340942,
  0.10101096332073212,
  -0.030797747895121574,
  -0.08900266885757446,
  0.0706370398402214,
  -0.05802832543849945,
  0.06717582046985626,
  -0.034788742661476135,
  -0.07374507188796997,
  0.03450619429349899,
  0.04326518625020981,
  -0.05258927494287491,
  0.02505848929286003,
  0.05107057839632034,
  -0.01721777766942978,
  -0.003147762967273593,
  -0.022091733291745186,
  0.01910731941461563,
  -0.02458168938755989,
  0.04488983750343323,
  -0.06671668589115143,
  -0.02103217877447605,
  0.10447218269109726,
  -0.03870909661054611,
  0.025658903643488884,
  0.02348681539297104,
  -0.052801188081502914,
  0.

Let't put the embedding vectors into our original dataframe

Convert embedding vectors list into a Pyspark DataFrame

In [11]:
features_column_names = [f"embedding_{i}" for i in range(len(embedding_vectors[0]))]
embeddings_df = spark.createDataFrame(embedding_vectors, schema=features_column_names)
embeddings_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

Add unique `row_id` to each row in the pysaprk dataframe

In [12]:
emdedding_df = embeddings_df.repartition(1).withColumn("row_id", F.monotonically_increasing_id())
emdedding_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

Add unique `row_id` to each row in our main pyspark dataframe `df`

In [13]:
df = df.repartition(1).withColumn("row_id", F.monotonically_increasing_id())
df.show()

+----------+--------------------+--------------------+--------------------+------+
|product_id|               title|         description|       combined_text|row_id|
+----------+--------------------+--------------------+--------------------+------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|Men's 3X Large Ca...|     0|
|        P1|Turmode 30 ft. RP...|If you need more ...|Turmode 30 ft. RP...|     1|
|        P2|Large Tapestry Bo...|Polyester cover r...|Large Tapestry Bo...|     2|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|16-Gauge-Sinks Ve...|     3|
|        P4|Men's Crazy Horse...|This 9 in. black ...|Men's Crazy Horse...|     4|
|        P5|Mariana 6 ft. Mul...|With robust struc...|Mariana 6 ft. Mul...|     5|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|5 gal. #650C-2 Po...|     6|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|7/8 in. x 4-1/2 i...|     7|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|Ring Gold Bar Car...|     8|
|   

Let's join the two dataframes

In [14]:
df = df.join(emdedding_df, on="row_id", how = "inner").drop("row_id")
df.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

### Task 3 - Cluster products using K-means

Assemble the 512 Embedding Columns into a Single 'features' Column

In [15]:
df = df.drop("features")

assembler = VectorAssembler(
    inputCols=features_column_names,
    outputCol="features"
)

data = assembler.transform(df)

data.select("product_id", "title", "description","features").show()

+----------+--------------------+--------------------+--------------------+
|product_id|               title|         description|            features|
+----------+--------------------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[0.04266477003693...|
|        P1|Turmode 30 ft. RP...|If you need more ...|[0.04413316026329...|
|        P2|Large Tapestry Bo...|Polyester cover r...|[0.04236160591244...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.0497337169945...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[0.02608588151633...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[0.06058844178915...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[0.00749773625284...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|[-0.0217796657234...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|[-4.6415856922976...|
|        P9|Traditional Silve...|This transitional...|[0.03908809646964...|
|       P10|

Apply K-Means Clustering with 5 Clusters on the `features` Column

In [16]:
kmeans = KMeans(k=5, featuresCol= 'features', predictionCol= "cluster")

model = kmeans.fit(data)
clustered_data = model.transform(data)
clustered_data.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

### Task 4 - Visualize the clusters

Let's reduce the dimensionality of our features for visualization purpose

`512 dimensions => 2 dimensions`

In [17]:
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(clustered_data)
pca_results = pca_model.transform(clustered_data)
pca_results.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

In [18]:
pca_df = pca_results.select("product_id", "pca_features","cluster").toPandas()
pca_df['x'] = pca_df['pca_features'].apply(lambda x: x[0])
pca_df['y'] = pca_df['pca_features'].apply(lambda x: x[1])
pca_df

,product_id,pca_features,cluster,x,y
0,P0,"[0.1886705570390783, 0.03427629720202379]",0,0.188671,0.034276
1,P1,"[-0.17395293084291785, -0.13300977704391637]",0,-0.173953,-0.133010
2,P2,"[-0.020221270225769545, 0.31632254404688487]",4,-0.020221,0.316323
3,P3,"[0.007379061990622163, 0.05933598090026088]",1,0.007379,0.059336
4,P4,"[-0.024039188902044394, -0.04418841276085854]",0,-0.024039,-0.044188
...,...,...,...,...,...
1995,P1995,"[0.15092489862554184, 0.29628170929700137]",1,0.150925,0.296282
1996,P1996,"[-0.05664715358883182, 0.5847812548294024]",4,-0.056647,0.584781
1997,P1997,"[0.1078786699953137, -0.004729741204862443]",2,0.107879,-0.004730
1998,P1998,"[0.6915255706878283, 0.09694229748007542]",3,0.691526,0.096942


Let's plot the Clusters

In [19]:
def plot_clusters(pca_df, num_clusters=5):
    """
    Plots a 2D visualization of clusters using Plotly Express.

    Parameters:
    - pca_df (DataFrame): A Pandas DataFrame containing columns 'x', 'y', and 'cluster'.
      'x' and 'y' are the 2D PCA components, and 'cluster' indicates the cluster label.
    - num_clusters (int): The number of unique clusters to display.
    - recently_viewed_df (DataFrame, optional): DataFrame with 'x' and 'y' coordinates for recently viewed products.

    This function creates an interactive scatter plot where each point is colored according to its cluster.
    Recently viewed products are marked as black crosses if provided.

    Returns:
    - fig (Figure): The Plotly figure object for the plot.
    """

    # Create the base cluster plot
    fig = px.scatter(
        pca_df,
        x='x',
        y='y',
        opacity=0.6,
        size_max=4,
        color= pca_df.cluster.astype(str),
        title='2D Visualization of Clusters with Recently Viewed Products',
        labels={'x': 'PCA Component 1', 'y': 'PCA Component 2'},
        category_orders={'cluster': list(range(num_clusters))},
        # show the product id in the tooltip
        hover_data={'product_id': True}

    )

    # Update layout to add legend title and adjust plot settings
    fig.update_layout(legend_title_text='Clusters', legend=dict(x=1, y=1), width=600, height=500)

    return fig

fig = plot_clusters(pca_df)
fig.show()

### Task 5 - Highlight recently viewed products

In [20]:
print("The user has recently viewed the following products: ", recently_viewed_products)

The user has recently viewed the following products:  ['P316', 'P333', 'P1115', 'P1691', 'P1082', 'P397', 'P1441', 'P1054']


Let's have a look at the records in our `clustered_data` dataframe related to the recently viewed products.

In [22]:
filtered_data =clustered_data.where(clustered_data.product_id.isin(recently_viewed_products))
unique_clusters = filtered_data.select("cluster").distinct().rdd.flatMap(lambda x: x).collect()
print("The user has recently viewed the following clusters: ", unique_clusters)

The user has recently viewed the following clusters:  [4, 3]


### Task 6 - Recommend products based on recently viewed products

Let's have a look at the recently viewed products titles

In [23]:
filtered_data.select("title").rdd.flatMap(lambda x: x).collect()

["Mystic Fitz Roy Beige 9' 0 x 12' 0 Area Rug",
 'Florida Shag Beige/Multi 3 ft. x 5 ft. Floral Area Rug',
 '1 gal. #M250-3 Apple Turnover Extra Durable Flat Interior Paint & Primer',
 '1 gal. #HDPG60 Misty Emerald Lake Flat Interior Paint and Primer',
 '1 qt. #S220-7 Molasses Extra Durable Flat Interior Paint & Primer',
 'Modern Gray/Multi 9 ft. x 12 ft. Vibrant Abstract Polyester Area Rug',
 '1 qt. #PPU6-06 Honey Locust Eggshell Enamel Low Odor Interior Paint & Primer',
 'Genet Rust/Red-Brown 8 ft. x 11 ft. Abstract Wool Area Rug']

Let's see the distinct clusters of the recenetly viewed products.

In [24]:
print(unique_clusters)

[4, 3]


Let's find the possible products for the recommendation.

In [28]:
possible_recommendations =clustered_data.filter(clustered_data.cluster.isin(unique_clusters)).filter(~clustered_data.product_id.isin(recently_viewed_products)).show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

Let's perform a groupby and generate a list of product IDs that can be recommended for each of the clusters.

In [39]:
recommendations =  clustered_data.groupby("cluster").agg(F.collect_list("product_id").alias("recommendations"))
recommendations_df = recommendations.toPandas()
recommendations_df['random_recommendations'] = recommendations_df['recommendations'].apply(lambda x: np.random.choice(x,5, replace=False).tolist())
recommendations_df.head()

,cluster,recommendations,random_recommendations
0,0,"[P0, P1, P4, P7, P13, P22, P23, P28, P37, P46,...","[P773, P1941, P1106, P1332, P281]"
1,4,"[P2, P21, P52, P71, P87, P101, P108, P119, P12...","[P1974, P927, P269, P1805, P1927]"
2,1,"[P3, P5, P8, P9, P14, P17, P20, P27, P29, P32,...","[P973, P147, P1236, P162, P471]"
3,3,"[P6, P11, P16, P18, P24, P26, P30, P33, P40, P...","[P1515, P6, P560, P1580, P377]"
4,2,"[P10, P12, P15, P19, P25, P31, P38, P41, P45, ...","[P1893, P1958, P1539, P1900, P1696]"


In [40]:
# write a python function to display the recommendations
def display_recommendations(row):
  # find the title of the product in df
  product_ids = row['random_recommendations']
  cluster = row.cluster

  titles = data. \
          filter(data["product_id"]. \
          isin(product_ids)).select("title").collect()

  print("\n")
  print("Recommendations for Cluster:", cluster)
  for title in titles:
    print(title[0])

recommendations_df.apply(display_recommendations, axis=1)



Recommendations for Cluster: 0
50-Watt Equivalent PAR20 Dimmable E26 Base Flood Lighting 90 Plus CRI LED Light Bulb in Warm White 3000K (6-Pack)
Julienne Peeler
Disc Brake Pad Set
65-Watt Equivalent E26 Energy Saving, Wet-Rated CFL Light Bulb 2700 K (3-Pack)
Brake Hydraulic Hose - Rear Right


Recommendations for Cluster: 4
Natural Fiber Marble/Dark Gray 3 ft. x 5 ft. Border Area Rug
Abstract Blue/Ivory 8 ft. x 10 ft. Area Rug
57 Grand by Nicole Curtis Machine Washable Multicolor 2 ft. x 4 ft. Bordered Transitional Area Rug
Tuareg Oasis Multi 8' 0 x 8' 0 Round Rug
Andromeda Multicolor Geometric 2 ft. x 3 ft. Scatter Area Rug


Recommendations for Cluster: 1
Buffalo Porcelain Sculpture
6 in. x 10 in. x 26 in. Western Red Cedar Alpine Rough Sawn Corbel
4 in. Aged Bronze Flush Mount Self-Adhesive House Number 6
Olton 23 in. 3-Light Polished Nickel Bath/ Vanity Light with Clear Glass
Gold Eternal Loop Metal Sculpture


Recommendations for Cluster: 3
5 gal. #650C-2 Powdery Mist Semi-Gloss

,0
0,None
1,None
2,None
3,None
4,None
